<a href="https://colab.research.google.com/github/ManuMerlo/AReS-Anesthesia-Response-Simulator/blob/main/python/notebooks/ARes_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Patient Simulator Notebook

This notebook allows you to run python code using individual units called **cells**.

### How to Run Cells

To execute a cell in Google Colab:

1. **Select the cell**: Click on the cell you want to run.
2. **Run the cell**: Press **Shift + Enter** or click the **Play button** on the left side of the cell.

The code will be executed, and any results—whether text, data, or visual output—will appear directly beneath the cell.

### Key Cells: Environment Setup

Before running other parts of the notebook, certain cells need to be executed to properly set up the environment. These include:

- **Code upload cells**: These cells load code from external sources (like OneDrive), unzip files, and set the correct working directory. Make sure to run these cells first to ensure access to the code files. [Cells #1, #2, #3]
  
- **Library installation cells**: These cells install or import necessary Python libraries required for the notebook to function properly. [Cells #4, #5]

- **Code setup cells**: These initialize important variables into the notebook. Run them to ensure the rest of the notebook has access to the needed variables.

### Important Note on Variables Initialization

If you modify any cell that initializes variables, you **must re-run that cell** so that other cells in the notebook can recognize the changes. Otherwise, the rest of the notebook will still reference the old values, and the updates will not be reflected.

### Visual Output and Plotting

When your code generates visual outputs (such as plots or graphs), they will automatically appear directly below the respective cell. If you want to save any visual outputs, simply click the download button next to the image to save it to your local machine.

# New Section

In [ ]:
# External library import
!pip install control


In [ ]:
# Local libraries import
from python.AReS import Simulator, SimulatorMode, Model, Interaction, DoHMeasure, TciMode

import numpy as np
import matplotlib.pyplot as plt

# from patient_simulator import TciMode
# from patient_simulator.utils.enums import DisturbanceType, SimulatorMode, Model, DoHMeasure, Interaction, VolumeStatus, PatientPhase
# from patient_simulator.simulator import Simulator

In [ ]:
# Global Variables for the Patient
# Change here to set the variables for the simulation:

patient_id = 0                        # This parameter must be in the range [0, 44]

t_sim = 60*30                         # This parameter set the duration of the entire simulation in [sec]

t_s = 5                               # This is the sampling time

opiates = True;                       # This parameter describe wether opiates are administered together with Propofol.
                                      # The admissible values are: True / False

blood_sampling = 'arterial'           # The admissible values are: 'arterial' / 'venous'


# This parameter specifies the pharmacokinetic models for the drugs
# The syntax is the following:
#     pk_models = { model }
# A model if defined as:
#     key : value
# where key can assume the values: prop, remi, nore, rocu
# 'prop' can be associated to Model.Schnider / Model.Eleveld
# 'remi' can be associated to Model.Minto/ Model.Eleveld
# 'nore' can be associated to Model.Joachim (Default)
# 'rocu' can be associated to Model.Dahe (Default)

pk_models = {'prop': Model.Eleveld, 'remi': Model.Eleveld}

# This parameter specifies the pharmacodynamic models for the drugs
# The syntax is the following:
#     pd_models = { model }
# A model if defined as:
#     key : value
# where 'key' can assume the values: prop, remi
# 'prop' can be associated to Model.Wav / Model.Schnider / Model.Eleveld
# 'remi' can be associated to Model.Minto/ Model.Eleveld

pd_models = {'prop': Model.Wav, 'remi': Model.Eleveld}

interaction = Interaction.Surface     # This parameter describe whether the interaction between propofol and remifentnail should be considered
                                      # The admissible values are: Interaction.No_interaction / Interaction.Surface

doh_measure = DoHMeasure.Both         # This parameter describe whether you want to compute the WAV, the BIS or Both (for computational reasons)
                                      # The admissible values are: DoHMeasure.WAV / DoHMeasure.BIS / DoHMeasure.Both

# This parameter is used to add surgical stimuli to the patient.
# The syntax is the following:
#     stimuli = { distubance }

# A disturbance if defined as:
#     start_time : ( DisturbanceType, duration, [ max_delta_doh, max_delta_co] )

# For example, '1 * 60: (DisturbanceType.INTUBATION, 5 * 60, [10, 3])' specifies that:
#     at time 60 [1 min],
#     an Intubation Disturbance is added,
#     it lasts 300 [5 min]
#     the max value for the doh distubance is 10
#     the max value for the co disturbance is 3

# The admissible values for the disturbance type are:  DisturbanceType.INTUBATION, DisturbanceType.INCISION, DisturbanceType.SKIN_MANIPULATION, DisturbanceType.SUTURE

# Every definition of a distubance, need to be followed by a ',' if another disturbance follows.

# If you do not want to add stimuli, just write: stimuli = None

stimuli = {
      1 * 60: (DisturbanceType.INTUBATION, 5 * 60, [10, 3]),
      15 * 60: (DisturbanceType.SKIN_MANIPULATION, 9 * 60, [5, 3]),
      10 * 60: (DisturbanceType.INCISION, 5 * 60, [15, 3]),
      25 * 60: (DisturbanceType.SUTURE, 3 * 60, [2, 2])
  }

# This parameter is used to modify the blood volume status of the patient.
# The syntax is the following:
#     volume_status = { status }

# A status if defined as:
#     start_time : (Status Type)

# For example, '20*60: VolumeStatus.Hypovolemia' specifies that:
#     at time 20*60 [20 min],
#     the state of the patient is set to Hypovolemic

# The admissible values for the status are:  VolumeStatus.Hypovolemia, VolumeStatus.Normovolemia, VolumeStatus.Hypervolemia

# Every definition of a state, need to be followed by a ',' if another state follows.

# If you do not want to change the status of the patient, just write: volume_status = None

volume_status = {
        10*60: VolumeStatus.Hypovolemia,
        15*60: VolumeStatus.Normovolemia,
        20*60: VolumeStatus.Hypervolemia
    }


# This parameters are used to specify the infusion rates of Propofol, Remifentanil, Norepinephrine and Rocuronium
# Examples:
#     u_prop = [0.2] * t_sim  specifies that a constant infusion rate of 0.2 should be used during the complete simulation
#     u_prop = [0.2] * 10 * 60 + [0.4] * 20 * 60 specifies that a constant infusion rate of 0.2 should be used during the first 10 minutes, and a constant infusion rate of 0.4 should be used during the last 20 minutes
# NOTE: The lenght of this array, must be equal to t_sim

u_prop = [0.2] * t_sim
u_remi = [0.1] * t_sim
u_nore = [0.02] * 10 * 60 + [0.04] * 20 * 60
u_rocu = [0.02] * t_sim


In [ ]:
# Gloval variables for the TCI

# This parameters specify the pharmacokinetic and pharmacodynamic models for the drugs in the TCI
# This allows the user to specify different models for the TCI and for the patient eventhought the availbale models are the same
pk_models_TCI = {'prop': Model.Schnider, 'remi': Model.Minto}
pd_models_TCI = {'prop': Model.Schnider, 'remi': Model.Minto}


# This parameter specifies the administration mode for the TCI.
# The user can specify the mode for for each drug.
# The syntax is the following:
#     modes_TCI = { mode }
# A mode if defined as:
#     key : value
# where key can assume the values: prop, remi, nore, rocu
# 'prop' can be associated to TciMode.EffectSite/ TciMode.Plasma
# 'remi' can be associated to TciMode.EffectSite/ TciMode.Plasma
# 'nore' can be associated only to TciMode.Plasma
# 'rocu' can be associated to TciMode.EffectSite/ TciMode.Plasma
# Every definition of a mode, need to be followed by a ',' if another mode follows.
# If you do not want to specify any mode, just write: modes_TCI = None.

modes_TCI = {'prop': TciMode.EffectSite,
             'remi': TciMode.Plasma,
             'rocu': TciMode.EffectSite}

# This parameter specifies the limits for the TCI.
# The user can specify the limits for the plasma concentrations and the infusion rates for each drug using the followig keys:
#       'cp_limit_prop'   'infusion_limit_prop'
#       'cp_limit_remi'   'infusion_limit_remi'
#       'cp_limit_nore'   'infusion_limit_nore'
#       'cp_limit_nore'   'infusion_limit_rocu'
# The syntax is the following:
#     limits_TCI = { limit }
# A limit if defined as:
#     key : value
# Every definition of a limit, need to be followed by a ',' if another limit follows.
# If you do not want to specify any limit, just write: limits_TCI = None

limits_TCI = {'cp_limit_prop': 5, 'cp_limit_remi': 5,
              'cp_limit_nore': 5, 'cp_limit_rocu': 5}

# This parameters are used to specify the target (plasma or effect site) concentrations of Propofol, Remifentanil, Norepinephrine and Rocuronium
# Examples:
#     target_prop = [4] * t_sim specifies that a constant concentration of 4 should be used during the complete simulation
#     u_prop = [4] * 10 * 60 + [2] * 20 * 60 specifies that a constant concentration of 2 should be used during the first 10 minutes, and a constant concentration of 4 should be used during the last 20 minutes
# NOTE: The lenght of this array, must be equal to t_sim

target_prop = [4] * t_sim
target_remi = [2] * t_sim
target_nore = [0] * t_sim
target_rocu = [5] * t_sim

In [ ]:
# Run this cell to simulate the patient with the patient id specified in the Global variable of the patient, section using infusion rates
# NOTE: Always run the cell with the Global Parameters the patient beforehand

simulator = Simulator.create()

simulator.init_simulation_from_file(id_patient=patient_id,
                                    t_sim=t_sim,
                                    t_s=t_s,
                                    pk_models=pk_models,
                                    pd_models=pd_models,
                                    interaction=interaction,
                                    doh_measure=doh_measure,
                                    stimuli=stimuli,
                                    volume_status=volume_status)

simulator.run_complete_simulation(u_prop, u_remi, u_nore, u_rocu)

simulator.plot_simulation()

In [ ]:
# Run this cell to simulate all the patients (from 0 to 4) using infusion rates
# NOTE: Always run the cell with the Global Parameters beforehand
# NOTE: you can run all the patients (from 0 to 45) but it takes about 10/15 minutes

initial_patient = 0 # this index is included in the range
final_patient = 4   # this index it not included in the range. Example: range(0,4) = [0,1,2,3]

simulator = Simulator.create()
for k in range(initial_patient, final_patient):
  simulator.init_simulation_from_file(id_patient=k,
                                      t_sim=t_sim,
                                      t_s=t_s,
                                      pk_models=pk_models,
                                      pd_models=pd_models,
                                      interaction=interaction,
                                      doh_measure=doh_measure,
                                      stimuli=stimuli,
                                      volume_status=volume_status)

  simulator.run_complete_simulation(u_prop, u_remi, u_nore, u_rocu)

simulator.plot_simulation()

In [ ]:
# Run this cell to simulate the patient with the patient id specified in the Global variable of the patient using the TCI system
# NOTE: Always run the cell with the Global Parameters for both the patient and the TCI beforehand

simulator = Simulator.create(SimulatorMode.Concentration)

simulator.init_simulation_from_file(id_patient=patient_id,
                                    t_sim=t_sim,
                                    t_s=t_s,
                                    pk_models=pk_models,
                                    pd_models=pd_models,
                                    interaction=interaction,
                                    doh_measure=doh_measure,
                                    stimuli=stimuli,
                                    volume_status=volume_status,
                                    pk_models_TCI=pk_models_TCI,
                                    pd_models_TCI=pd_models_TCI,
                                    limits_TCI=limits_TCI,
                                    modes_TCI=modes_TCI)

simulator.run_complete_simulation(target_prop, target_remi, target_nore, target_rocu)

simulator.plot_simulation()

In [ ]:
# Run this cell to simulate the patients (from 0 to 4) using the TCI system
# NOTE: Always run the cell with the Global Parameters for both the patient and the TCI beforehand
# NOTE: you can run all the patients (from 0 to 45) but it takes about 10/15 minutes

initial_patient = 0
final_patient = 4

simulator = Simulator.create(SimulatorMode.Concentration)
for k in range(initial_patient, final_patient):

  simulator.init_simulation_from_file(id_patient=k,
                                      t_sim=t_sim,
                                      t_s=t_s,
                                      pk_models=pk_models,
                                      pd_models=pd_models,
                                      interaction=interaction,
                                      doh_measure=doh_measure,
                                      stimuli=stimuli,
                                      volume_status=volume_status,
                                      pk_models_TCI=pk_models_TCI,
                                      pd_models_TCI=pd_models_TCI,
                                      limits_TCI=limits_TCI,
                                      modes_TCI=modes_TCI)

  simulator.run_complete_simulation(target_prop, target_remi, target_nore, target_rocu)

simulator.plot_simulation()

In [ ]:
from scipy.optimize import differential_evolution
from multiprocessing import Pool, cpu_count

# Create a global pool of worker processes
pool = Pool(processes=cpu_count())

def simulate_patient(args):
    # Your existing `simulate_patient` function
    k, kp, ki, kd, pid_setpoint, t_s, t_sim, pk_models, pd_models, interaction, dohMeasure = args
    total_iae = 0
    # Initialize PID controller and simulator for the patient
    pid = PID(kp=kp, ki=ki, kd=kd, setpoint=pid_setpoint, sample_time=t_s, output_limits=(0, 2))
    simulator = Simulator()
    simulator.init_simulation_from_file(id_patient=k, t_sim=t_sim, t_s=t_s, pk_models=pk_models,
                                        pd_models=pd_models,
                                        interaction=interaction, doh_measure=dohMeasure)
    i = 0
    print(f"running simulation patient {k}, kp {kp}, ki {ki}, kd {kd}")
    # Run simulation for the induction phase
    while simulator.get_patient_phase() == PatientPhase.Induction and i < t_sim // t_s:
        wav = simulator.get_patient_state()['WAV']
        u_prop = pid.compute(wav / 100, t_s)
        simulator.one_step_simulation(u_prop, u_prop / 2, 0, 0)
        i += 1
        error = abs(pid.setpoint - wav / 100)
        total_iae += error * t_s  # Accumulate IAE
    return total_iae

def objective(params):
    kp, ki, kd = params
    pid_setpoint = 0.5
    t_sim = 30 * 60  # Total simulation time in seconds
    t_s = 5          # Time step in seconds
    interaction = Interaction.Surface
    dohMeasure = DoHMeasure.Both
    pk_models = {'prop': Model.Eleveld, 'remi': Model.Eleveld}
    pd_models = {'prop': Model.Wav, 'remi': Model.Eleveld}
    # List of patient IDs to simulate
    patient_ids = range(40, 45)
    # Prepare arguments for parallel execution
    args = [
        (
            k, kp, ki, kd, pid_setpoint, t_s, t_sim,
            pk_models, pd_models, interaction, dohMeasure
        )
        for k in patient_ids
    ]
    # Use the global pool to parallelize patient simulations
    results = pool.map(simulate_patient, args)
    # Sum the total IAE from all patients
    total_iae = sum(results)
    return total_iae

def optimize_pid():
    bounds = [(0.3, 0.3), (0.02, 0.02), (0.3, 0.3)]
    # Use differential evolution to find optimal parameters
    result = differential_evolution(
        objective,
        bounds,
        strategy='best1bin',
        maxiter=5,   # Reduce iterations
        popsize=5,   # Reduce population size
        tol=0.01,    # Enable early stopping based on relative tolerance
        atol=0.001,  # Enable early stopping based on absolute tolerance
        workers=1    # Single thread to prevent nested parallelism issues
    )
    # Extract the best kp, ki, kd values
    best_kp, best_ki, best_kd = result.x
    print(f"Optimal PID parameters: kp={best_kp}, ki={best_ki}, kd={best_kd}")

# Ensure the global pool is closed properly
try:
    optimize_pid()
finally:
    pool.close()
    pool.join()
